In [1]:
%%capture
# 安装编译依赖
!sudo apt-get update -y
!sudo apt-get install -y wget build-essential zlib1g-dev libncurses5-dev \
    libgdbm-dev libnss3-dev libssl-dev libreadline-dev libffi-dev curl libbz2-dev

# 下载 Python 3.12.8 源码
!wget https://www.python.org/ftp/python/3.12.10/Python-3.12.10.tgz
!tar -xf Python-3.12.10.tgz
%cd Python-3.12.10

# 编译并安装（altinstall 避免覆盖系统自带 python3）
!./configure --enable-optimizations
!make -j 4
!sudo make altinstall

# 安装 pip
!wget https://bootstrap.pypa.io/get-pip.py
!python3.12 get-pip.py

# 确认版本
!python3.12 --version

In [2]:
%cd /kaggle/working/
!git clone https://github.com/ProjectNeura/MIPCandy.git
%cd /kaggle/working/MIPCandy/
!python3.12 -m pip install -q -e .
!python3.12 -m pip install -q pyyaml torchvision monai pyvista
!python3.12 -m pip install -q torch==2.6.0 torchvision --index-url https://download.pytorch.org/whl/cu126

/kaggle/working
Cloning into 'MIPCandy'...
remote: Enumerating objects: 486, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 486 (delta 207), reused 136 (delta 125), pack-reused 229 (from 3)
Receiving objects: 100% (486/486), 1.68 MiB | 30.13 MiB/s, done.
Resolving deltas: 100% (273/273), done.
/kaggle/working/MIPCandy
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mipcandy (pyproject.toml) ... done


In [3]:
%cd /kaggle/working/MIPCandy/
!git clone https://github.com/ProjectNeura/mipcandy-bundles.git
%cd /kaggle/working/MIPCandy/mipcandy-bundles/
!python3.12 -m pip install -q -e .

/kaggle/working/MIPCandy
Cloning into 'mipcandy-bundles'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 61 (delta 14), reused 5 (delta 3), pack-reused 37 (from 1)
Receiving objects: 100% (61/61), 19.94 KiB | 4.98 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/kaggle/working/MIPCandy/mipcandy-bundles
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mipcandy-bundles (pyproject.toml) ... done


In [4]:
%cd /kaggle/working/
code = """
import os
os.environ['MPLBACKEND'] = 'Agg'

import torch
from torch.utils.data import DataLoader
from mipcandy.data import NNUNetDataset
from mipcandy_bundles.unet import UNetTrainer
from mipcandy import NotionFrontend
from monai.transforms import Compose, SpatialCrop, CenterSpatialCrop, MapLabelValue

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # device = "cpu"
    data_dir = "/kaggle/input/brats2020-nnunet/nnUNet_raw/Dataset320_BRaTS" # 240*240*155

    spatial_transforms = Compose([
        SpatialCrop(roi_start=(5, 0, 0), roi_end=(149, 240, 240)),
        CenterSpatialCrop(roi_size=(144, 96, 96))
    ]) # depth [5:149] -> 144, then center crop to 96x96
    label_transforms = Compose([
        spatial_transforms,
        MapLabelValue(orig_labels=[0, 1, 2, 4], target_labels=[0, 1, 2, 3])
    ])
    train_dataset, val_dataset = NNUNetDataset(folder=data_dir, split="Tr",
                                               image_transform=spatial_transforms,
                                               label_transform=label_transforms).fold(fold=0)
    train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    trainer = UNetTrainer("./test_output_BRaTS", train_dataloader, val_dataloader, device=device)
    trainer.num_dims = 3
    # trainer.set_frontend(NotionFrontend)
    trainer.num_classes = 4
    # trainer.sliding_window_shape = (48, 48, 48)
    trainer.train(num_epochs=100, note="BRaTS", early_stop_tolerance=10)
"""
with open("content.py", 'w') as f:
    f.write(code)

!python3.12 content.py

/kaggle/working
[2025-11-30 23:44:08.052985] Experiment (ID 20251130-23-fa38) created at 
2025-11-30 23:44:08.052886
[2025-11-30 23:44:08.055948] Trainer: UNetTrainer
[2025-11-30 23:44:08.056464] Note: BRaTS
[2025-11-30 23:44:08.058227] Set to manual seed 89
[2025-11-30 23:44:08.901532] Example input shape: (4, 144, 96, 96)
[2025-11-30 23:44:09.381341] Model: UNet
[2025-11-30 23:44:09.382024] MACs: 301.9 G / Params: 15.6 M
[2025-11-30 23:44:09.382549] Example output shape: (4, 144, 96, 96)
Training epoch 1 (0.1486) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00  
[2025-11-30 23:49:38.382011] Training combined loss: 0.2000 @[0.2000, 0.2000] 
(N/A)
[2025-11-30 23:49:38.383596] Training soft dice: 0.8793 @[0.8793, 0.8793] (N/A)
[2025-11-30 23:49:38.384177] Training bce loss: 0.1585 @[0.1585, 0.1585] (N/A)
[2025-11-30 23:49:38.384780] Training learning rate: 0.0100 @[0.0100, 0.0100] 
(N/A)
                    Epoch 1 Training                    
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━